
# HW6 Diffusion Model

**Sources:**
- Github implementation [Denoising Diffusion Pytorch](https://github.com/lucidrains/denoising-diffusion-pytorch)
- Papers on Diffusion models ([Dhariwal, Nichol, 2021], [Ho et al., 2020] ect.)


## Import Packages and Set Seeds

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!apt-get install git-lfs
!git lfs install
!apt-get install unzip
!git clone https://huggingface.co/datasets/LeoFeng/MLHW_6
!unzip ./MLHW_6/faces.zip -d .
# install required dependencies
!pip install einops
!pip install transformers
!pip install ema_pytorch
!pip install accelerate

In [ ]:
!pip install matplotlib
!pip install stylegan2_pytorch

In [ ]:
import math
import copy
from pathlib import Path
from random import random
from functools import partial
from collections import namedtuple
from multiprocessing import cpu_count

import torch
from torch import nn, einsum
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torch.optim import Adam

import torchvision
from torchvision import transforms as T, utils

from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange

from PIL import Image
from tqdm.auto import tqdm
from ema_pytorch import EMA

from accelerate import Accelerator
import matplotlib.pyplot as plt
import os

torch.backends.cudnn.benchmark = True
torch.manual_seed(4096)

if torch.cuda.is_available():
  torch.cuda.manual_seed(4096)

## Step 1: Forward process (Noise scheduler)




In [ ]:
def linear_beta_schedule(timesteps):
    """
    linear schedule, proposed in original ddpm paper
    """
    scale = 1000 / timesteps
    beta_start = scale * 0.0001
    beta_end = scale * 0.02
    return torch.linspace(beta_start, beta_end, timesteps, dtype = torch.float64)

def extract(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))

Create dataset

In [ ]:
class Dataset(Dataset):
    def __init__(
        self,
        folder,
        image_size
    ):
        self.folder = folder
        self.image_size = image_size
        self.paths = [p for p in Path(f'{folder}').glob(f'**/*.jpg')]
        #################################
        ## TODO: Data Augmentation ##
        #################################
        self.transform = T.Compose([
            T.Resize(image_size),
            T.RandomHorizontalFlip(),
            T.RandomRotation(90),
            T.RandomGrayscale(p=0.2),
            T.ToTensor(),
        ])
        # self.transform = T.Compose([
        #     T.Resize(image_size),
        #     T.ToTensor()
        # ])

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        path = self.paths[index]
        img = Image.open(path)
        return self.transform(img)

# StyleGAN2_Pytorch

In [ ]:
# reference from stylegan2-pytorch(https://github.com/lucidrains/stylegan2-pytorch)
!stylegan2_pytorch --data './faces' --image-size 64 --batch-size 16 --num-train-steps 20000

In [ ]:
!stylegan2_pytorch --generate --num_generate 340 --num_image_tiles 1 --results_dir "./"
!mv default submission

In [ ]:
import os
path = './submission'
i = 1
for file in os.listdir(path):
    old_path = os.path.join(path, file)
    new_path = os.path.join(path, f'{i}.jpg')
    os.rename(old_path, new_path)
    i += 1
print(f'Have {i} imgs')

In [ ]:
import os
folder_path = './submission'
files = os.listdir(folder_path)
for file in files:
    file_number = int(file.split('.')[0])
    if file_number > 1000:
        file_path = os.path.join(folder_path, file)
        os.remove(file_path)

In [ ]:
%cd ./submission
!tar -zcf ../submission.tgz *.jpg
%cd ..

# Q1

In [ ]:
!mkdir ./q1_faces
!mv ./faces/10.jpg ./faces/11.jpg ./faces/12.jpg ./faces/13.jpg ./faces/14.jpg ./q1_faces

In [ ]:
#!stylegan2_pytorch --data './q1_faces/14.jpg' --generate --load-from 0 --results_dir "./q1" --num_image_tiles 1
#!stylegan2_pytorch --data './q1/default/generated-04-17-2023_07-36-20-0-mr.jpg' --generate --load-from 1 --results_dir "./q1" --num_image_tiles 1
#!stylegan2_pytorch --data './q1/default/generated-04-17-2023_07-36-55-0-mr.jpg' --generate --load-from 2 --results_dir "./q1" --num_image_tiles 1
#!stylegan2_pytorch --data './q1/default/generated-04-17-2023_07-37-12-0-mr.jpg' --generate --load-from 3 --results_dir "./q1" --num_image_tiles 1
#!stylegan2_pytorch --data './q1/default/generated-04-17-2023_07-37-28-0-mr.jpg' --generate --load-from 4 --results_dir "./q1" --num_image_tiles 1
#!stylegan2_pytorch --data './q1/default/generated-04-17-2023_07-37-44-0-mr' --generate --load-from 5 --results_dir "./q1" --num_image_tiles 1
#!stylegan2_pytorch --data './q1/default/generated-04-17-2023_07-38-04-0-mr.jpg' --generate --load-from 6 --results_dir "./q1" --num_image_tiles 1
#!stylegan2_pytorch --data './q1/default/generated-04-17-2023_07-38-20-0-mr.jpg' --generate --load-from 7 --results_dir "./q1" --num_image_tiles 1
!stylegan2_pytorch --data './q1/default/generated-04-17-2023_07-38-37-0-mr.jpg' --generate --load-from 8 --results_dir "./q1" --num_image_tiles 1